In [ ]:
import pickle
import os, glob

import numpy as np

import matplotlib.pyplot as plt

## Load Data and Extract Events

In [ ]:
directory = 'INSERT_DIRECTORY' # Load simulated data created by "CreateData.ipynb"
files = glob.glob(os.path.join(directory, '*.pkl'))
files

In [ ]:
molecules = [x.split('_')[-2] for x in files] # Extract class names
molecules

In [ ]:
def extract_events(data):
    traces = data[0]
    evt_locs = data[1]
    
    events = []
    for i, trace in enumerate(evt_locs):
        for event in trace:
            start = event[0]
            end = event[1]

            event = traces[start:end, i]
            events.append(event)
    return events
        
all_data = []
data_events = []
for i in range(len(molecules)):
    molecule = molecules[i]
    
    print(molecule)
    
    file = files[i]
    with open(file, 'rb') as f:
        data = pickle.load(f)
    all_data.append(data)
    
    events = extract_events(data)
    data_events.append(events)

## Pad Events
Ensures each event has the same length by padding with zeros. This allows use within a neural network.

In [ ]:
longest_evt = max([len(x) for molecule in data_events for x in molecule])
longest_evt   

In [ ]:
data_events_padded = []
for i, molecule in enumerate(data_events):
    molecule_events_padded = []
    for j, event in enumerate(molecule):
        new_event = np.pad(event, (longest_evt - len(event), 0), mode='constant') 
        molecule_events_padded.append(new_event)
    data_events_padded.append(molecule_events_padded)

## Save Padded Events

In [ ]:
for i, molecule in enumerate(molecules):
    fname = 'sim_{}_events.pkl'.format(molecule)
    padded_events = data_events_padded[i]
    
    with open(fname, 'wb') as f:
        pickle.dump(padded_events, f)
    

## Display Example Event

In [ ]:
i = 0 #molecule idx
j = 0 #trace idx
k = 1 #event idx
pad = 500

trace = all_data[i][0][:, j]
x = np.linspace(0, len(trace), len(trace))
evt = all_data[i][1][j][k]

fig, ax = plt.subplots()
ax.plot(x, trace)
ax.plot(x[evt[0]:evt[1]], trace[evt[0]:evt[1]], color='red')

ax.set(xlim=(evt[0] - pad, evt[1] + pad))

Plot examples from each class in a single figure

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(6, 4), dpi=600, sharey=True)
pad = 100
j = 1
k = 3

order = [1, 4, 3, 0, 2]
labels = ['(a)', '(b)', '(c)', '(d)', '(e)']

for i, idx in enumerate(order):
    ax = axs.flatten()[i]
    trace = all_data[idx][0][:, j]
    x = np.linspace(0, len(trace), len(trace)) / 25
    evt = all_data[idx][1][j][k]
    ax.plot(x[evt[0]-pad:evt[1]+pad], trace[evt[0]-pad:evt[1]+pad])
    ax.plot(x[evt[0]:evt[1]], trace[evt[0]:evt[1]], color='red')

    ax.set_xlabel("Time / ms", weight='bold')
    ax.annotate(labels[i], xy=(0.0, 1.07), xytext=(0, 0), xycoords='axes fraction', textcoords='offset points', weight='bold')
    ax.tick_params(width=1.5)
    for axis in ['top', 'right', 'bottom', 'left']:
        ax.spines[axis].set_linewidth(1.5)
    # ax.set_title(molecules[idx], weight='bold')
axs[0, 0].set_ylabel("Current / nA", weight='bold')
axs[1, 0].set_ylabel("Current / nA", weight='bold')
axs[-1, -1].remove()
    
fig.tight_layout()
# fig.savefig("Example Sim Evts.png")